In [1]:
#=================================================================================================#
# Modeling the data

# Main Metric
#.         Precision
#.         Precision

#=================================================================================================#
import os
import pandas as pd
from pathlib import Path
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score,accuracy_score,precision_score,recall_score
from sklearn.preprocessing import OneHotEncoder,LabelEncoder,MinMaxScaler,StandardScaler
# from tensorflow.keras.callbacks import ReduceLROnPlateau

# Set styling for better visualization
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Global configuration
DATA_ROOT=Path("../data/interim/").resolve()
DATASET_DIR=Path.joinpath(DATA_ROOT,'creditcard.csv')


In [2]:
# Load the data
def load_dataset(path_dir):
    df=pd.read_csv(path_dir,sep=",")
    return df

df=load_dataset(DATASET_DIR)
X=df.drop('Class',axis=1)
y=df['Class'] # Target
# Split data
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

#Scaling data
scaler=StandardScaler()
X_train_scaled=scaler.fit_transform(X_train)
X_test_scaled=scaler.fit(X_test)


In [3]:
# Process the target# 
y.value_counts()


Class
0    284315
1       492
Name: count, dtype: int64

In [26]:
# Create a simple model
def create_simple_model(input_shape):
    model=tf.keras.Sequential(
        [
            tf.keras.layers.Input(shape=(input_shape,1)),
            tf.keras.layers.Dense(64,activation="relu"),
            tf.keras.layers.Dropout(0.3),
            tf.keras.layers.Dense(32,activation="relu"),
            tf.keras.layers.Dropout(0.3),
            tf.keras.layers.Dense(16,activation="relu"),
            tf.keras.layers.Dense(1,activation="sigmoid")
        ]
        
    )

    #compile the model
    model.compile(
        optimizer= tf.keras.optimizers.Adam(learning_rate=initial_lr),
        loss='binary_crossentropy',
        metrics=['accuracy', tf.keras.metrics.AUC()]
        
    )
    return  model
# Create a simple model
def create_fairly_medium_model(input_shape):
    model=tf.keras.Sequential(
        [
            tf.keras.layers.Input(shape=(input_shape,1)),
            tf.keras.layers.Dense(64,activation="relu"),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Dropout(0.3),

            tf.keras.layers.Dense(32,activation="relu"),
            tf.keras.layers.Dense(2,activation="softmax")
        ]
        
    )

    #compile the model
    model.compile(
        optimizer= tf.keras.optimizers.Adam(),
        loss="sparse_categorical_crossentropy",
        metrics=['precision']
        
    )
    return  model

    # Create a simple model
def create_advanced_model(input_shape):
    model=tf.keras.Sequential(
        [
            tf.keras.layers.Input(shape=(input_shape,1)),
            tf.keras.layers.Dense(128,activation="relu"),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Dropout(0.3),

            tf.keras.layers.Dense(64,activation="relu"),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Dropout(0.3),

            tf.keras.layers.Dense(64,activation="relu"),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Dropout(0.3),

            tf.keras.layers.Dense(32,activation="relu"),
            tf.keras.layers.Dense(32,activation="relu"),
            tf.keras.layers.Dropout(0.2),
            tf.keras.layers.Dense(2,activation="softmax")
        ]
        
    )

    #compile the model
    model.compile(
        optimizer= tf.keras.optimizers.Adam(),
        loss='binary_crossentropy',
        metrics=['accuracy', tf.keras.metrics.AUC()]
        
    )
    return  model

# setting callbacks functions
def set_callbacks():
        checkpoint_dir = 'credit_card_checkpoints'
        if not os.path.exists(checkpoint_dir):
          os.makedirs(checkpoint_dir)
            
        callbacks=[tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,  # Reduce LR by half when plateauing
        patience=10,  # Wait for 10 epochs before reducing LR
        min_delta=0.0001,
        min_lr=1e-6,
        verbose=1
    ),
        tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=25,  # Increased patience to allow LR reduction to take effect
        min_delta=0.0001,
        mode='min',
        restore_best_weights=True,
        verbose=1
    ),
        tf.keras.callbacks.ModelCheckpoint(
        filepath=os.path.join(checkpoint_dir, 'best_model.keras'),
        monitor='val_loss',
        mode='min',
        save_best_only=True,
        verbose=1
    )
            
        ]
        return callbacks

In [27]:
X_train_scaled.shape[0]

227845

In [28]:
y_train.shape

(227845,)

In [29]:
model=create_simple_model(X_train_scaled.shape[1])

NameError: name 'initial_lr' is not defined

In [25]:
    # Train model
    history = model.fit(
        X_train_scaled, y_train,
        epochs=200,
        batch_size=64,
        validation_split=0.2,
        callbacks=set_callbacks(),
        verbose=1
    )
    
    
#     return model, history, X_test, y_test

Epoch 1/200


ValueError: Arguments `target` and `output` must have the same shape. Received: target.shape=(None, 1), output.shape=(None, 30)

In [ ]:
# def plot_training_history(history):
#     """Plot training curves including learning rate changes"""
#     fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(20, 5))
    
#     # Plot loss
#     ax1.plot(history.history['loss'], label='Training Loss')
#     ax1.plot(history.history['val_loss'], label='Validation Loss')
#     ax1.set_title('Model Loss')
#     ax1.set_xlabel('Epoch')
#     ax1.set_ylabel('Loss (MSE)')
#     ax1.legend()
#     ax1.grid(True)
    
#     # Plot MAE
#     ax2.plot(history.history['mae'], label='Training MAE')
#     ax2.plot(history.history['val_mae'], label='Validation MAE')
#     ax2.set_title('Model Mean Absolute Error')
#     ax2.set_xlabel('Epoch')
#     ax2.set_ylabel('MAE')
#     ax2.legend()
#     ax2.grid(True)
    
#     # Plot Learning Rate
#     ax3.plot(history.history['learning_rate'], label='Learning Rate')
#     ax3.set_title('Learning Rate Over Time')
#     ax3.set_xlabel('Epoch')
#     ax3.set_ylabel('Learning Rate')
#     ax3.set_yscale('log')  # Use log scale for better visualization
#     ax3.legend()
#     ax3.grid(True)
    
#     plt.tight_layout()
#     plt.show()

In [ ]:
# # Results Evaluation function
# def evaluate_results(model, X_test, y_test, history):
#     """Evaluate and print model results"""
#     # Evaluate on test set
#     test_loss, test_mae = model.evaluate(X_test, y_test, verbose=0)
    
#     # Get training statistics
#     best_epoch = np.argmin(history.history['val_loss']) + 1
#     total_epochs = len(history.history['loss'])
#     best_val_loss = min(history.history['val_loss'])
#     best_val_mae = min(history.history['val_mae'])
    
#     # Get learning rate statistics
#     initial_lr = history.history['learning_rate'][0]
#     final_lr = history.history['learning_rate'][-1]
#     num_lr_drops = sum(1 for i in range(1, len(history.history['learning_rate']))
#                       if history.history['learning_rate'][i] < history.history['learning_rate'][i-1])
    
#     print("\nModel Performance Summary:")
#     print("-" * 50)
#     print(f"Best validation loss achieved at epoch: {best_epoch}/{total_epochs}")
#     print(f"Best validation loss (MSE): {best_val_loss:.4f}")
#     print(f"Best validation MAE: {best_val_mae:.4f}")
#     print(f"Test loss (MSE): {test_loss:.4f}")
#     print(f"Test MAE: {test_mae:.4f}")
#     print("\nLearning Rate Summary:")
#     print(f"Initial learning rate: {initial_lr:.6f}")
#     print(f"Final learning rate: {final_lr:.6f}")
#     print(f"Number of learning rate reductions: {num_lr_drops}")
#     print("-" * 50)